In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

Not a perfect match.  But could add other bookies back in and
would maybe have more matchups btwn games_df and odds.  Would
porbably want to take average odds before running simulation.
Also would want to ignore games where there are no odds available.

Would probably need to match on last name, first letter, and maybe
match month or something.

In [2]:
# Read tennis odds from Kaggle Dataset

df = pd.read_csv('t_odds.csv')

In [3]:
# Read the DataFrame constructed as game_df.

gdf = pd.read_pickle('oct_20.pkl')

In [4]:
# Get just the betting odds from bet365, for season 2016
# Extension 1: fix 2017 data in games_df module to use 2017 as holdout
# Extension 2: use more than just Bet365 odds -- avg available bookies' odds?

df = df[['date', 'player_1_name', 'player_2_name', 'doubles',
        'bet365_payout', 'bet365_player_1_odd', 'bet365_player_2_odd']].dropna()
df = df.loc[df.doubles == 0]
df.date = pd.to_datetime(df.date)
df = df.loc[(df.date >= datetime.date(month=1, year=2016, day=1)) &
            (df.date <= datetime.date(month=1, year=2017, day=1))]

In [5]:
# Get just the rows from game_df in saeson 2016.

gdf.tourney_date = pd.to_datetime(gdf.tourney_date)
gdf = gdf.loc[(gdf.tourney_date >= datetime.date(month=1, year=2016, day=1)) & 
             (gdf.tourney_date <= datetime.date(month=1, year=2017, day=1))]

In [6]:
# Helper fxn.

def convert_name(full_name):
    """
    Convert First/Last name combo to Last Name, First Initial.
    E.g. Roger Federer --> Federer R.
    """
    x = full_name.split()
    return x[1].title() + " " + x[0][0].upper() + "."

In [7]:
# Now apply name conversion to all of gdf.

gdf['player1_name'] = gdf.player1_name.apply(convert_name)
gdf['player2_name'] = gdf.player2_name.apply(convert_name);

In [8]:
# Helper fxn.

def implied_probability(decimal_odds):
    """
    Convert a player's decimal odds to implied probabilty.
    """
    
    return (1/decimal_odds) * 100

In [9]:
# Convert decimal odds to implied probabilities.

df['player_1_implied_probability'] = df.bet365_player_1_odd.apply(implied_probability)
df['player_2_implied_probability'] = df.bet365_player_2_odd.apply(implied_probability)

In [10]:
# Get just the month-year info from gdf.tourney_date, and ignore the rest.\
# day is set to equal 1 in all cases: this is ok for my purposes (i think)

gdf['merge_date'] = gdf.tourney_date.astype(str).apply(lambda x: x[:7])
gdf.merge_date = gdf.merge_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%M').date())

# Do the same for df.date.

df['merge_date'] = df.date.astype(str).apply(lambda x: x[:7])
df.merge_date = df.merge_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%M').date())

In [11]:
#### The case where player1 / player2 tuples DO match up.

# Create a new column to merge on: tuple of player names.

# Create new column
df['players_forward'] = df.player_1_name + "," + df.player_2_name
gdf['players_forward'] = gdf.player1_name + "," + gdf.player2_name

# Split into list object
gdf.players_forward = gdf.players_forward.apply(lambda x: x.split(','))
df.players_forward = df.players_forward.apply(lambda x: x.split(','))

# Convert to tuple.
gdf.players_forward = gdf.players_forward.apply(lambda x: tuple(x))
df.players_forward = df.players_forward.apply(lambda x: tuple(x))

In [12]:
#### The case where player1 / player2 tuples DO NOT match up.

# Create a new column to merge on: tuple of player names.

# Create new column
df['players_backward'] = df.player_2_name + "," + df.player_1_name
gdf['players_backward'] = gdf.player1_name + "," + gdf.player2_name

# Split into list object
gdf.players_backward = gdf.players_backward.apply(lambda x: x.split(','))
df.players_backward = df.players_backward.apply(lambda x: x.split(','))

# Convert to tuple.
gdf.players_backward = gdf.players_backward.apply(lambda x: tuple(x))
df.players_backward = df.players_backward.apply(lambda x: tuple(x))

In [19]:
# Create merged DataFrames.

forward_match = gdf.merge(df, on=['merge_date', 'players_forward'])
backward_match = gdf.merge(df, on=['merge_date', 'players_backward'])

In [20]:
# Switch the p1 and p2 probabilities in backward_match, to correct for the misordering.

actual_p1_prob = backward_match.player_2_implied_probability.copy(deep=True)
actual_p2_prob = backward_match.player_1_implied_probability.copy(deep=True)
backward_match.player_1_implied_probability = actual_p1_prob
backward_match.player_2_implied_probability = actual_p2_prob

In [21]:
# DataFrames ready to concat and reorder based on tourney_date.

games_df = pd.concat([forward_match, backward_match])

In [22]:
### May need to add more columns -- don't remember off the top if my head what exactly went into games_df before.

game = ['game_index', 'label', 'player_1_implied_probability', 'player_2_implied_probability']

deltas = ['delta_1stIn_expw', 'delta_1stIn_mavg', 'delta_1stWon_expw',
'delta_1stWon_mavg', 'delta_2ndWon_expw', 'delta_2ndWon_mavg',
'delta_SvGms_expw', 'delta_SvGms_mavg', 'delta_ace_expw',
'delta_ace_mavg', 'delta_age', 'delta_bpFaced_expw', 'delta_bpFaced_mavg',
'delta_bpSaved_expw', 'delta_bpSaved_mavg', 'delta_df_expw',
'delta_df_mavg', 'delta_ht', 'delta_rank', 'delta_rank_points',
'delta_svpt_expw', 'delta_svpt_mavg']

cols = game + deltas
games_df = games_df[cols]
games_df = games_df.sort_values('game_index')

In [ ]:
# Final Product: Games DF, with player1 and play2 implied probabilties
# There are dupes: where did they get in, how to fix?

games_df

### Investigation of Duplicates

In [13]:
df.head()

,date,player_1_name,player_2_name,doubles,bet365_payout,bet365_player_1_odd,bet365_player_2_odd,player_1_implied_probability,player_2_implied_probability,merge_date,players_forward,players_backward
8,2016-10-24 11:30:00,Saez-Royuela Ariza A.,Pokotilov B.,0.0,93.2,12.00,1.01,8.333333,99.009901,2016-01-01,"(Saez-Royuela Ariza A., Pokotilov B.)","(Pokotilov B., Saez-Royuela Ariza A.)"
9,2016-09-15 13:10:00,Santillan A.,Giannessi A.,0.0,92.5,2.25,1.57,44.444444,63.694268,2016-01-01,"(Santillan A., Giannessi A.)","(Giannessi A., Santillan A.)"
16,2016-09-21 12:30:00,De Greef A.,Andujar-Alba P.,0.0,93.0,2.37,1.53,42.194093,65.359477,2016-01-01,"(De Greef A., Andujar-Alba P.)","(Andujar-Alba P., De Greef A.)"
30,2016-05-11 01:10:00,Abduraimova N.,Naydenova A.,0.0,93.0,1.53,2.37,65.359477,42.194093,2016-01-01,"(Abduraimova N., Naydenova A.)","(Naydenova A., Abduraimova N.)"
34,2016-08-29 18:50:00,Mertens E.,Muguruza G.,0.0,95.9,11.00,1.05,9.090909,95.238095,2016-01-01,"(Mertens E., Muguruza G.)","(Muguruza G., Mertens E.)"


In [18]:
gdf.head()

,surface,tourney_level,tourney_date,game_index,player1_name,player1_seed,player1_ht,player1_age,player1_rank,player1_rank_points,...,delta_svpt_expw,delta_1stIn_expw,delta_1stWon_expw,delta_2ndWon_expw,delta_SvGms_expw,delta_bpSaved_expw,delta_bpFaced_expw,merge_date,players_forward,players_backward
83636,Hard,A,2016-01-04 00:01:00,83636,Raonic M.,1.0,34.0,196.000000,4.0,6.0,...,-415.349969,-421.540881,-446.128368,-586.427313,-309.362990,-141.024001,-227.583888,2016-01-01,"(Raonic M., Federer R.)","(Raonic M., Federer R.)"
83637,Hard,A,2016-01-04 00:01:00,83637,Thiem D.,NaN,1600.0,2.000000,4.0,6.0,...,-557.639931,-620.445640,-623.467768,-721.304368,-624.834373,-439.027609,-385.700136,2016-01-01,"(Thiem D., Federer R.)","(Thiem D., Federer R.)"
83638,Hard,A,2016-01-04 00:01:00,83638,Tomic B.,193.0,2.0,3.000000,7.0,41.0,...,-110.577935,-145.367501,-212.940825,-121.504799,-295.701352,-253.178126,-148.574172,2016-01-01,"(Tomic B., Raonic M.)","(Tomic B., Raonic M.)"
83639,Hard,A,2016-01-04 00:01:00,83639,Federer R.,0.0,1.0,34.406571,3.0,7.0,...,530.039207,563.583433,576.993398,635.121225,428.223401,304.611612,122.021193,2016-01-01,"(Federer R., Dimitrov G.)","(Federer R., Dimitrov G.)"
83640,Hard,A,2016-01-04 00:01:00,83640,Thiem D.,NaN,1600.0,0.000000,2.0,6.0,...,-26.103870,-109.315892,-47.043038,-122.361150,-223.618277,-140.544133,-118.662238,2016-01-01,"(Thiem D., Cilic M.)","(Thiem D., Cilic M.)"


In [27]:
len(gdf.game_index) == len(gdf.game_index.unique())

True

`games_df` looks like it's got duplicated games.  This probably happened because `df` and `gdf` had some players that played each other multiple times in a month.

In [24]:
games_df.head()

,game_index,label,player_1_implied_probability,player_2_implied_probability,delta_1stIn_expw,delta_1stIn_mavg,delta_1stWon_expw,delta_1stWon_mavg,delta_2ndWon_expw,delta_2ndWon_mavg,...,delta_bpFaced_mavg,delta_bpSaved_expw,delta_bpSaved_mavg,delta_df_expw,delta_df_mavg,delta_ht,delta_rank,delta_rank_points,delta_svpt_expw,delta_svpt_mavg
0,83636,1,25.000000,80.000000,-421.540881,-1694.380713,-446.128368,-179.435965,-586.427313,-303.433607,...,-9380.822040,-141.024001,-703.742857,-334.090537,-1263.811111,-0.406571,-30.0,1.0,-415.349969,313.444444
3,83637,0,14.285714,90.909091,-620.445640,-2155.941783,-623.467768,-467.981367,-721.304368,-904.909461,...,-9359.838467,-439.027609,-3414.226133,-412.332237,-1189.944444,1599.000000,1.0,0.0,-557.639931,-372.070994
2,83637,0,28.571429,76.923077,-620.445640,-2155.941783,-623.467768,-467.981367,-721.304368,-904.909461,...,-9359.838467,-439.027609,-3414.226133,-412.332237,-1189.944444,1599.000000,1.0,0.0,-557.639931,-372.070994
6,83638,0,26.666667,78.125000,-145.367501,-251.710857,-212.940825,-261.249624,-121.504799,-515.093695,...,24.877429,-253.178126,-2890.252357,-72.668393,304.600000,-194.000000,-33.0,29.0,-110.577935,-626.819202
7,83638,0,38.167939,66.666667,-145.367501,-251.710857,-212.940825,-261.249624,-121.504799,-515.093695,...,24.877429,-253.178126,-2890.252357,-72.668393,304.600000,-194.000000,-33.0,29.0,-110.577935,-626.819202


In [30]:
# There are 1521 - 1284 = 237 duped games.

len(games_df.game_index), len(games_df.game_index.unique())

(1521, 1284)

Look at `game_index == 83637` as an example; this is the first duped game in `games_df`

In [38]:
gdf[gdf.game_index == 83637].merge_date

83637    2016-01-01
Name: merge_date, dtype: object

In [40]:
df[(df.player_2_name == "Thiem D.") & (df.player_1_name == "Federer R.")]

,date,player_1_name,player_2_name,doubles,bet365_payout,bet365_player_1_odd,bet365_player_2_odd,player_1_implied_probability,player_2_implied_probability,merge_date,players_forward,players_backward
118896,2016-06-11 06:25:00,Federer R.,Thiem D.,0.0,94.8,1.3,3.5,76.923077,28.571429,2016-01-01,"(Federer R., Thiem D.)","(Thiem D., Federer R.)"
122269,2016-01-09 00:10:00,Federer R.,Thiem D.,0.0,95.1,1.1,7.0,90.909091,14.285714,2016-01-01,"(Federer R., Thiem D.)","(Thiem D., Federer R.)"


Something is going wrong with `merge_date`.  Above, the two Thiem v Federer games merged into a dupe, because they both have the same `merge_date` value.  However, the first game should have a value of 01-2016 and the second should have 06-2016, but they both wound up with a 01-2016 value.  This looks like the root of the problem!

Autopsy of the `merge_date` creation on `df`.

In [ ]:
# # Get just the month-year info from gdf.tourney_date, and ignore the rest.\
# # day is set to equal 1 in all cases: this is ok for my purposes (i think)

# gdf['merge_date'] = gdf.tourney_date.astype(str).apply(lambda x: x[:7])
# gdf.merge_date = gdf.merge_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%M').date())

# # Do the same for df.date.

# df['merge_date'] = df.date.astype(str).apply(lambda x: x[:7])
# df.merge_date = df.merge_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%M').date())

In [48]:
test_df = pd.read_csv('t_odds.csv')
test_df = test_df[['date', 'player_1_name', 'player_2_name', 'doubles',
        'bet365_payout', 'bet365_player_1_odd', 'bet365_player_2_odd']].dropna()
test_df = test_df.loc[test_df.doubles == 0]
test_df.date = pd.to_datetime(test_df.date)
test_df = test_df.loc[(test_df.date >= datetime.date(month=1, year=2016, day=1)) &
            (test_df.date <= datetime.date(month=1, year=2017, day=1))]

In [49]:
test_df = test_df.head(10)
test_df

,date,player_1_name,player_2_name,doubles,bet365_payout,bet365_player_1_odd,bet365_player_2_odd
8,2016-10-24 11:30:00,Saez-Royuela Ariza A.,Pokotilov B.,0.0,93.2,12.00,1.01
9,2016-09-15 13:10:00,Santillan A.,Giannessi A.,0.0,92.5,2.25,1.57
16,2016-09-21 12:30:00,De Greef A.,Andujar-Alba P.,0.0,93.0,2.37,1.53
30,2016-05-11 01:10:00,Abduraimova N.,Naydenova A.,0.0,93.0,1.53,2.37
34,2016-08-29 18:50:00,Mertens E.,Muguruza G.,0.0,95.9,11.00,1.05
35,2016-06-14 11:45:00,Viola M.,Ymer E.,0.0,93.7,3.50,1.28
36,2016-08-26 02:00:00,Opelka R.,Giraldo S.,0.0,92.7,2.10,1.66
44,2016-10-05 13:30:00,Wagner B.,Van Den Berg B.,0.0,92.2,1.16,4.50
56,2016-05-05 03:10:00,Wang Y.,Peng S.,0.0,92.7,2.10,1.66
68,2016-09-27 14:25:00,Struff J-L.,Medvedev D.,0.0,92.5,1.72,2.00


In [44]:
# test_df['merge_date'] = test_df.date.astype(str).apply(lambda x: x[:7])
# test_df.merge_date = test_df.merge_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%M').date())

In [50]:
test_df['merge_date'] = test_df.date.astype(str).apply(lambda x: x[:7])
test_df

,date,player_1_name,player_2_name,doubles,bet365_payout,bet365_player_1_odd,bet365_player_2_odd,merge_date
8,2016-10-24 11:30:00,Saez-Royuela Ariza A.,Pokotilov B.,0.0,93.2,12.00,1.01,2016-10
9,2016-09-15 13:10:00,Santillan A.,Giannessi A.,0.0,92.5,2.25,1.57,2016-09
16,2016-09-21 12:30:00,De Greef A.,Andujar-Alba P.,0.0,93.0,2.37,1.53,2016-09
30,2016-05-11 01:10:00,Abduraimova N.,Naydenova A.,0.0,93.0,1.53,2.37,2016-05
34,2016-08-29 18:50:00,Mertens E.,Muguruza G.,0.0,95.9,11.00,1.05,2016-08
35,2016-06-14 11:45:00,Viola M.,Ymer E.,0.0,93.7,3.50,1.28,2016-06
36,2016-08-26 02:00:00,Opelka R.,Giraldo S.,0.0,92.7,2.10,1.66,2016-08
44,2016-10-05 13:30:00,Wagner B.,Van Den Berg B.,0.0,92.2,1.16,4.50,2016-10
56,2016-05-05 03:10:00,Wang Y.,Peng S.,0.0,92.7,2.10,1.66,2016-05
68,2016-09-27 14:25:00,Struff J-L.,Medvedev D.,0.0,92.5,1.72,2.00,2016-09


In [57]:
test_df.merge_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m').date())

8     2016-10-01
9     2016-09-01
16    2016-09-01
30    2016-05-01
34    2016-08-01
35    2016-06-01
36    2016-08-01
44    2016-10-01
56    2016-05-01
68    2016-09-01
Name: merge_date, dtype: object

The problem was that I was using '%M' where I should have used '%m' to target the month with the .strptime() function. Looks like `gdf` has the same problem:

In [60]:
gdf.merge_date.head()

83636    2016-01-01
83637    2016-01-01
83638    2016-01-01
83639    2016-01-01
83640    2016-01-01
Name: merge_date, dtype: object

To fix this, I will copy this notebok and make the %m modification to the merge_date column creation.